# Object Detection Demo
Based on Tensorflow Object Detection. Detecting Traffic and More

# Imports

In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO

from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

if StrictVersion(tf.__version__) < StrictVersion('1.9.0'):
  raise ImportError('Please upgrade your TensorFlow installation to v1.9.* or later!')


In [2]:
import cv2
# import time
cap = cv2.VideoCapture("Traffic 02.mp4")
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
writer = cv2.VideoWriter('Traffic Detection23.mp4', cv2.VideoWriter_fourcc(*'XVID'),13, (width, height))

## Env setup

In [3]:
# This is needed to display the images.
%matplotlib inline
import matplotlib.pyplot as plt

## Object detection imports
Here are the imports from the object detection module.

In [4]:
from utils import label_map_util

from utils import visualization_utils as vis_util

# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing `PATH_TO_FROZEN_GRAPH` to point to a new .pb file.  

By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

In [5]:
# What model to download.
MODEL_NAME = 'faster_rcnn_resnet101_coco_11_06_2017'
# MODEL_FILE = MODEL_NAME + '.tar.gz'
# DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
# PATH_TO_FROZEN_GRAPH = MODEL_NAME + '/frozen_inference_graph.pb'
PATH_TO_FROZEN_GRAPH = os.path.join('faster_rcnn_inception_resnet_v2_atrous_coco_11_06_2017', 'frozen_inference_graph.pb')
# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')

## Download Model

In [6]:
# opener = urllib.request.URLopener()
# opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
# tar_file = tarfile.open(MODEL_FILE)
# for file in tar_file.getmembers():
#   file_name = os.path.basename(file.name)
#   if 'frozen_inference_graph.pb' in file_name:
#     tar_file.extract(file, os.getcwd())

## Load a (frozen) Tensorflow model into memory.

In [7]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [8]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)
print(category_index)

{1: {'id': 1, 'name': 'person'}, 2: {'id': 2, 'name': 'bicycle'}, 3: {'id': 3, 'name': 'car'}, 4: {'id': 4, 'name': 'motorcycle'}, 5: {'id': 5, 'name': 'airplane'}, 6: {'id': 6, 'name': 'bus'}, 7: {'id': 7, 'name': 'train'}, 8: {'id': 8, 'name': 'truck'}, 9: {'id': 9, 'name': 'boat'}, 10: {'id': 10, 'name': 'traffic light'}, 11: {'id': 11, 'name': 'fire hydrant'}, 13: {'id': 13, 'name': 'stop sign'}, 14: {'id': 14, 'name': 'parking meter'}, 15: {'id': 15, 'name': 'bench'}, 16: {'id': 16, 'name': 'bird'}, 17: {'id': 17, 'name': 'cat'}, 18: {'id': 18, 'name': 'dog'}, 19: {'id': 19, 'name': 'horse'}, 20: {'id': 20, 'name': 'sheep'}, 21: {'id': 21, 'name': 'cow'}, 22: {'id': 22, 'name': 'elephant'}, 23: {'id': 23, 'name': 'bear'}, 24: {'id': 24, 'name': 'zebra'}, 25: {'id': 25, 'name': 'giraffe'}, 27: {'id': 27, 'name': 'backpack'}, 28: {'id': 28, 'name': 'umbrella'}, 31: {'id': 31, 'name': 'handbag'}, 32: {'id': 32, 'name': 'tie'}, 33: {'id': 33, 'name': 'suitcase'}, 34: {'id': 34, 'name'

# Detection

In [9]:
with detection_graph.as_default():
  with tf.Session(graph=detection_graph) as sess:
    while True:
      ret, image_np = cap.read()
      # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
      image_np_expanded = np.expand_dims(image_np, axis=0)
      image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
      # Each box represents a part of the image where a particular object was detected.
      boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
      # Each score represent how level of confidence for each of the objects.
      # Score is shown on the result image, together with the class label.
      scores = detection_graph.get_tensor_by_name('detection_scores:0')
      classes = detection_graph.get_tensor_by_name('detection_classes:0')
      num_detections = detection_graph.get_tensor_by_name('num_detections:0')
    
      # Actual detection.
      (boxes, scores, classes, num_detections) = sess.run(
          [boxes, scores, classes, num_detections],
          feed_dict={image_tensor: image_np_expanded})
    
      # Visualization of the results of a detection.
        
      vis_util.visualize_boxes_and_labels_on_image_array(
          image_np,
          np.squeeze(boxes),
          np.squeeze(classes).astype(np.int32),
          np.squeeze(scores),
          category_index,
          use_normalized_coordinates=True,
          line_thickness=2)
      

      writer.write(image_np)
      cv2.imshow('object detection', cv2.resize(image_np, (1080,720)))
#       cv2.imshow('object detection', image_np)
      if cv2.waitKey(25) & 0xFF == ord('q'):
#           cv2.destroyAllWindows()
          break
#     cap.release()
cap.release()
writer.release()
cv2.destroyAllWindows()

['车辆:4', '皮卡车:1', '交通灯:5', '人:7', '长型车:1']
['车辆:4', '皮卡车:1', '交通灯:5', '长型车:1', '人:4', '狗:1']
['车辆:4', '皮卡车:1', '交通灯:5', '长型车:1', '人:3', '狗:1']
['车辆:4', '皮卡车:1', '交通灯:5', '人:8', '狗:1', '长型车:1']
['车辆:4', '皮卡车:1', '交通灯:5', '人:8', '长型车:1', '狗:1']
['车辆:4', '皮卡车:1', '交通灯:5', '人:7', '长型车:1', '狗:1']
['车辆:4', '皮卡车:1', '交通灯:5', '人:9', '长型车:1', '狗:1']
['车辆:4', '皮卡车:1', '长型车:1', '交通灯:5', '人:7']
['车辆:4', '皮卡车:1', '长型车:1', '人:6', '交通灯:5', '狗:1']
['车辆:4', '长型车:1', '皮卡车:1', '人:6', '交通灯:5', '狗:1']
['车辆:4', '长型车:1', '皮卡车:1', '人:6', '交通灯:5', '狗:1']
['车辆:5', '皮卡车:1', '交通灯:5', '人:8', '长型车:1']
['车辆:5', '皮卡车:1', '交通灯:5', '人:8', '长型车:1']
['车辆:6', '皮卡车:1', '人:9', '交通灯:5', '长型车:1', '狗:1']
['车辆:6', '皮卡车:1', '人:9', '交通灯:5', '长型车:1', '狗:1']
['车辆:6', '皮卡车:1', '人:9', '交通灯:5', '长型车:1', '狗:1']
['车辆:5', '皮卡车:1', '人:6', '交通灯:5', '长型车:1', '狗:1']
['车辆:4', '皮卡车:1', '长型车:1', '人:7', '交通灯:5', '狗:1']
['车辆:4', '皮卡车:1', '人:7', '交通灯:5', '长型车:1']
['车辆:6', '皮卡车:1', '人:6', '交通灯:5', '狗:1']
['车辆:4', '皮卡车:1', '交通灯:5', '人:6', '长型车:1']
[